In [8]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [9]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [10]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [11]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [12]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [13]:
file_parameters

{'watershed': 'nak_auto'}

In [14]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [15]:
pd.set_option('display.max_columns', 1000)

In [16]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [17]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [18]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [19]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)
#length = 1

In [20]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/nak/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['도개_2015.xlsx', '도개_2016.xlsx', '도개_2017.xlsx', '도개_2018.xlsx', '도개_2019.xlsx', '도개_2020.xlsx'], ['신암_2015.xlsx', '신암_2016.xlsx', '신암_2017.xlsx', '신암_2018.xlsx', '신암_2019.xlsx', '신암_2020.xlsx'], ['풍양_2015.xlsx', '풍양_2016.xlsx', '풍양_2017.xlsx', '풍양_2018.xlsx', '풍양_2019.xlsx', '풍양_2020.xlsx']]
data/nak/자동/도개_2015.xlsx
data/nak/자동/도개_2016.xlsx
data/nak/자동/도개_2017.xlsx
data/nak/자동/도개_2018.xlsx
data/nak/자동/도개_2019.xlsx
data/nak/자동/도개_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/nak/자동/신암_2015.xlsx
data/nak/자동/신암_2016.xlsx
data/nak/자동/신암_2017.xlsx
data/nak/자동/신암_2018.xlsx
data/nak/자동/신암_2019.xlsx
data/nak/자동/신암_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/nak/자동/풍양_2015.xlsx
data/nak/자동/풍양_2016.xlsx
data/nak/자동/풍양_201

1/1 [==============================] - 0s 21ms/step - gen_loss: 11.5750 - disc_loss: 0.3236 - rmse: 0.3870 - val_loss: 0.3200
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.1611 - disc_loss: 0.3352 - rmse: 0.3808 - val_loss: 0.4454
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0607 - disc_loss: 0.3254 - rmse: 0.3701 - val_loss: 0.3960
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7739 - disc_loss: 0.3081 - rmse: 0.3589 - val_loss: 0.3745
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7776 - disc_loss: 0.3165 - rmse: 0.3782 - val_loss: 0.3620
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4911 - disc_loss: 0.3058 - rmse: 0.3634 - val_loss: 0.3082
Epoch 56/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.5887 - disc_loss: 0.3029 - rmse: 0.4180 - val_loss: 0.3408
Epoch 57/2000
1/1 [====================

Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7275 - disc_loss: 0.2511 - rmse: 0.3918 - val_loss: 0.3349
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9724 - disc_loss: 0.2761 - rmse: 0.3389 - val_loss: 0.3129
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8049 - disc_loss: 0.2386 - rmse: 0.3068 - val_loss: 0.2751
Epoch 112/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.9341 - disc_loss: 0.2375 - rmse: 0.2816 - val_loss: 0.3027
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5199 - disc_loss: 0.2404 - rmse: 0.2789 - val_loss: 0.2966
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2717 - disc_loss: 0.2609 - rmse: 0.3406 - val_loss: 0.3266
Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9124 - disc_loss: 0.2751 - rmse: 0.3347 - val_loss: 0.3175
Epoch 116/2000
1/1 [

1/1 [==============================] - 0s 21ms/step - gen_loss: 6.0938 - disc_loss: 0.2308 - rmse: 0.3170 - val_loss: 0.2790
Epoch 168/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.8465 - disc_loss: 0.2250 - rmse: 0.2831 - val_loss: 0.3338
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4421 - disc_loss: 0.2345 - rmse: 0.2855 - val_loss: 0.3055
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9556 - disc_loss: 0.2295 - rmse: 0.3154 - val_loss: 0.2719
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6256 - disc_loss: 0.2281 - rmse: 0.3071 - val_loss: 0.2669
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4685 - disc_loss: 0.2419 - rmse: 0.3328 - val_loss: 0.2776
Epoch 173/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0713 - disc_loss: 0.2342 - rmse: 0.3332 - val_loss: 0.2951
Epoch 174/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1880 - disc_loss: 0.2287 - rmse: 0.2753 - val_loss: 0.2943
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5975 - disc_loss: 0.2306 - rmse: 0.2984 - val_loss: 0.3160
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5493 - disc_loss: 0.2316 - rmse: 0.2773 - val_loss: 0.3345
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7332 - disc_loss: 0.2287 - rmse: 0.2850 - val_loss: 0.2989
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4112 - disc_loss: 0.2155 - rmse: 0.2852 - val_loss: 0.3318
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5569 - disc_loss: 0.2204 - rmse: 0.2824 - val_loss: 0.2779
Epoch 231/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0816 - disc_loss: 0.2245 - rmse: 0.2679 - val_loss: 0.3515
Epoch 232/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8592 - disc_loss: 0.1649 - rmse: 0.2542 - val_loss: 0.2884
Epoch 1154/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7032 - disc_loss: 0.1739 - rmse: 0.2676 - val_loss: 0.2935
Epoch 1155/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9614 - disc_loss: 0.1652 - rmse: 0.2328 - val_loss: 0.2802
Epoch 1156/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1586 - disc_loss: 0.1541 - rmse: 0.2118 - val_loss: 0.2377
Epoch 1157/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 2.7500 - disc_loss: 0.1710 - rmse: 0.2186 - val_loss: 0.2339
Epoch 1158/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.8862 - disc_loss: 0.1651 - rmse: 0.2219 - val_loss: 0.2653
Epoch 1159/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6691 - disc_loss: 0.1502 - rmse: 0.2081 - val_loss: 0.2862
Epoch 1160/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8888 - disc_loss: 0.1535 - rmse: 0.2068 - val_loss: 0.2567
Epoch 1212/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8630 - disc_loss: 0.1640 - rmse: 0.2331 - val_loss: 0.2110
Epoch 1213/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0628 - disc_loss: 0.1488 - rmse: 0.2103 - val_loss: 0.2647
Epoch 1214/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0576 - disc_loss: 0.1640 - rmse: 0.2365 - val_loss: 0.2136
Epoch 1215/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1408 - disc_loss: 0.1559 - rmse: 0.2126 - val_loss: 0.2325
Epoch 1216/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6186 - disc_loss: 0.1499 - rmse: 0.2155 - val_loss: 0.2614
Epoch 1217/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.8375 - disc_loss: 0.1490 - rmse: 0.2500 - val_loss: 0.2363
Epoch 1218/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4050 - disc_loss: 0.1619 - rmse: 0.2307 - val_loss: 0.2323
Epoch 1270/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.2927 - disc_loss: 0.1472 - rmse: 0.2726 - val_loss: 0.2696
Epoch 1271/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.3259 - disc_loss: 0.1667 - rmse: 0.2432 - val_loss: 0.2307
Epoch 1272/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0219 - disc_loss: 0.1586 - rmse: 0.2412 - val_loss: 0.2229
Epoch 1273/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5704 - disc_loss: 0.1530 - rmse: 0.2652 - val_loss: 0.2444
Epoch 1274/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9160 - disc_loss: 0.1660 - rmse: 0.2234 - val_loss: 0.2180
Epoch 1275/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0994 - disc_loss: 0.1539 - rmse: 0.2529 - val_loss: 0.2389
Epoch 1276/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4113 - disc_loss: 0.1615 - rmse: 0.2580 - val_loss: 0.2194
Epoch 1328/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2150 - disc_loss: 0.1534 - rmse: 0.2373 - val_loss: 0.2532
Epoch 1329/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0768 - disc_loss: 0.1575 - rmse: 0.2157 - val_loss: 0.2723
Epoch 1330/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2220 - disc_loss: 0.1523 - rmse: 0.2234 - val_loss: 0.2414
Epoch 1331/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6283 - disc_loss: 0.1522 - rmse: 0.2140 - val_loss: 0.2923
Epoch 1332/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.3149 - disc_loss: 0.1661 - rmse: 0.2198 - val_loss: 0.2693
Epoch 1333/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3749 - disc_loss: 0.1530 - rmse: 0.2442 - val_loss: 0.2084
Epoch 1334/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5139 - disc_loss: 0.1495 - rmse: 0.2261 - val_loss: 0.2748
Epoch 1386/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5605 - disc_loss: 0.1589 - rmse: 0.3023 - val_loss: 0.2252
Epoch 1387/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8605 - disc_loss: 0.1561 - rmse: 0.2095 - val_loss: 0.2645
Epoch 1388/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7912 - disc_loss: 0.1581 - rmse: 0.2143 - val_loss: 0.2300
Epoch 1389/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9846 - disc_loss: 0.1575 - rmse: 0.2131 - val_loss: 0.2811
Epoch 1390/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0060 - disc_loss: 0.1582 - rmse: 0.2129 - val_loss: 0.2702
Epoch 1391/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1837 - disc_loss: 0.1543 - rmse: 0.2201 - val_loss: 0.2711
Epoch 1392/2000
1/1 [========

In [21]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [109]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [110]:
print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]


(52608, 36)
tp


In [111]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (36825, 36) val_df.shape :  (10522, 36) test_df.shape: (5261, 36)


In [112]:
#fake_df.shape[0]/8760

In [113]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

/home/jaewon/work/venv/water-quality-venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [116]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  6
out_num_features :  1


In [117]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [118]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/nak/models/phosphorus/


In [119]:
rnn_epochs

20

## 모델 학습

In [123]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/20
10/10 [==============================] - ETA: 0s - loss: 1.4006 - mean_absolute_error: 0.5002 - nse: 0.2271
Epoch 00001: val_loss improved from inf to 0.41944, saving model to save/best_model.h5
10/10 [==============================] - 8s 792ms/step - loss: 1.4006 - mean_absolute_error: 0.5002 - nse: 0.2271 - val_loss: 0.4194 - val_mean_absolute_error: 0.4552 - val_nse: 0.1204
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 2.0524 - mean_absolute_error: 0.3868 - nse: 0.3030
Epoch 00002: val_loss did not improve from 0.41944
10/10 [==============================] - 8s 771ms/step - loss: 2.0524 - mean_absolute_error: 0.3868 - nse: 0.3030 - val_loss: 0.5991 - val_mean_absolute_error: 0.5382 - val_nse: -0.2659
Epoch 3/20
10/10 [==============================] - ETA: 0s - loss: 1.9675 - mean_absolute_error: 0.3603 - nse: 0.3693
Epoch 00003: val_loss improved from 0.41944 to 0.32390, saving model to save/best_model.h5
10/10 [==============================] - 8s

## core / window.py / 

In [124]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.017401064
predict_day :  5
------------------------
0.00589368 8.261039e-05 0.009089026
nse3 :  [0.36801183]
pbias3 :  [-12.763842]
save/nak/models/phosphorus/
year : 2015 ~ 2020
run :  range(0, 1)
target :  tp
length :  52608
train=7, test=1, val=2
---------------
[0.36801183]
[12.763842]


In [ ]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************            
#                   SSSSSSSS               ******            ******              *****    *****                      
#        SSSS         S  SSS               ******            ******              *****    *****                     
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
